In [20]:
import pandas as pd
import numpy as np
import itertools
from scipy.stats import ttest_ind_from_stats
from sklearn.preprocessing import MinMaxScaler
import pickle
from scipy.cluster.hierarchy import dendrogram, linkage, is_valid_linkage
from matplotlib import pyplot as plt
%matplotlib inline

In [60]:
#load data from excel into dataframe
df=pd.read_excel('SV 2016 data edited 12202016 ver2.xlsx', sheetname='Sheet1')

In [ ]:
df.dtypes

In [25]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,216,217,218,219,220,221,222,223,224,225
0,Chrysler 200 Sdn,Chrysler,Mid-Size Car,Handling,SUV,Other (not listed),Married,Leading Edge Boomers (1946 - 1959),Some Post-Grad,"Teacher, Educator - Retired",...,0.076012,-1.122374,0.247485,-1.035740,-1.556995,-1.216453,-0.403176,-3.203542,-1.961525,0.996702
1,Chrysler 200 Sdn,Chrysler,Mid-Size Car,Overall driving performance,Sporty Car,Available financing or leasing terms,Partnered,Generation X (1965 - 1978),Some College,Technical Specialty,...,0.076012,-0.420489,0.247485,0.757939,0.269658,0.227776,-0.403176,-0.118321,-0.277746,-0.115697
2,Chrysler 200 Sdn,Chrysler,Mid-Size Car,Overall exterior workmanship,Larger Car,Price/deal offered,Married,Millennials (1979 - ),College Graduate,"Police, Postal, Fire",...,-0.769946,-1.122374,0.247485,-1.035740,1.182984,-1.216453,0.881167,0.910087,-1.119635,0.996702
3,Chrysler 200 Sdn,Chrysler,Mid-Size Car,Overall safety of the vehicle,Larger Car,Other (not listed),Married,Millennials (1979 - ),College Graduate,Health Care,...,0.921971,0.281396,0.987858,1.355832,-0.643669,0.227776,-0.403176,-0.118321,-0.277746,-0.115697
4,Chrysler 200 Sdn,Chrysler,Mid-Size Car,Price/deal offered,Smaller Car,Didn't like it / preferred vehicle bought,Single,Millennials (1979 - ),High School Graduate,Executive/Managerial,...,0.921971,0.983281,0.987858,0.160046,0.269658,0.949891,0.881167,0.910087,1.406034,0.996702


In [66]:
catList = [x for x in range(16)]
#makeList=['Acura','Audi','BMW','Buick','Cadillac','Chevrolet','Chrysler','Dodge','Ford','GMC','Honda','Hyundai','Infiniti','Jeep','Kia','Lexus','Lincoln','Mazda','Mercedes-Benz','MINI','Mitsubishi','Nissan','RAM','Subaru','Tesla','Toyota','Volkswagen','Volvo']
#optionExcludeList=['Kansas','Louisiana','Oklahoma','Arkansas','New Hampshire','Maine','New Mexico','Idaho','Mississippi','Rhode Island','West Virginia','Delaware','Nebraska','District Of Columbia','Montana','South Dakota','Vermont','North Dakota','Lincoln','Volvo','Infiniti','Mitsubishi','Mini','Land Rover','Fiat','Jaguar','Scion','Porsche','Mercury','Suzuki','American','RAM','Chevy','Bentley','Eagle','Daewoo','smart','Isuzu','Maserati','Alfa','Daihatsu','Ferrari','Aston','Peugeot','Oldsmobile','Plymouth','Mercedes-Benz','Saturn']

In [80]:
#initialize dfFull dict with full dataset
dfFull = {'Full': df.describe().T[['mean','std','count']]}
#add each subgroup dataframe to dfFull dict under subcategory key
for subset in catList:
    print(subset)
    for option in df[subset].dropna().unique():
        test=df[df[subset]==option].describe().T[['mean','std','count']]
        dfFull[str(subset)+'_'+option]=test

0


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [28]:
dfFull['Full'].head()

,mean,std,count
16,-3.291716e-15,1.0,27711.0
17,-8.449650e-16,1.0,27711.0
18,-1.343446e-14,1.0,27711.0
19,4.028894e-15,1.0,27711.0
20,-3.900793e-15,1.0,27711.0


In [68]:
keyList=dfFull.keys()
dfTest=pd.DataFrame(index=[dfFull['Full'].index])
for k in keyList:
    dfTest=pd.concat([dfTest,dfFull[k]['mean']],axis=1)

In [ ]:
keyList=list(dfFull.keys())
els=[]
comboList=[]
for s1, s2 in itertools.combinations(keyList,2):
    els.append([s1,s2])
    comboList.append(np.concatenate([dfFull[s1].values,dfFull[s2].values],axis=1))
    
len(els)

In [ ]:
els

In [ ]:
pickle.dump(dfFull, open( "full2.p", "wb" ))

In [ ]:
dfFull = pickle.load( open( "full2.p", "rb" ) )

In [ ]:
#iterate through each possible dataframe, concatenate new ttest frame to dfTest
#dfTest=pd.DataFrame(index=[dfFull['Full'].index], columns=['mean1','std1','count1', 'mean2','std2','count2', 'subset1', 'subset2'])
count=0
#for subsets in els:
#    count=count+1
#    if count%1000==0:
#        print(count)
dfTest=np.concatenate(comboList, axis=0)

In [73]:
dfTest.describe()

,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,...,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
count,2.100000e+02,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,...,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000
mean,2.356137e-16,-0.020373,0.049781,-0.112633,0.016049,0.183001,-0.073564,-0.116517,-0.084464,-0.194815,...,-0.060594,-0.054675,0.023780,0.001650,-0.081449,-0.003195,-0.088632,0.056939,-0.046105,0.102075
std,1.215390e-14,0.211516,0.307032,0.272746,0.412869,0.303357,0.350085,0.323148,0.236008,0.418140,...,0.095995,0.191281,0.200429,0.170904,0.171936,0.255467,0.128724,0.064725,0.048901,0.130256
min,-5.163187e-14,-0.646266,-0.659184,-0.938030,-1.253654,-0.607876,-1.123438,-1.073321,-0.934212,-1.384032,...,-0.265021,-0.414248,-0.513186,-0.671152,-1.010154,-1.008233,-0.531279,-0.144135,-0.170859,-0.350404
25%,-3.977265e-15,-0.144514,-0.166072,-0.263364,-0.233851,-0.044147,-0.290622,-0.277623,-0.212592,-0.420861,...,-0.128029,-0.182754,-0.117941,-0.122800,-0.177309,-0.147613,-0.165866,0.011792,-0.076851,0.016046
50%,-3.110235e-16,-0.007455,0.036307,-0.113637,-0.018381,0.159375,-0.098185,-0.128310,-0.083643,-0.241187,...,-0.063752,-0.082642,0.002474,0.002514,-0.079287,-0.031359,-0.081752,0.054651,-0.041318,0.089671
75%,3.597255e-15,0.100848,0.236345,0.080230,0.280759,0.395417,0.163661,0.051729,0.089143,0.014754,...,-0.019604,0.044830,0.171031,0.123908,0.007015,0.138840,-0.016674,0.103618,-0.006652,0.175122
max,6.367245e-14,0.856307,1.064721,0.584868,1.380724,1.345266,1.013983,0.834867,0.511678,1.092577,...,0.502494,0.682815,0.581755,0.495635,0.409119,0.892835,0.399101,0.251093,0.074294,0.517534


In [ ]:
t,p=ttest_ind_from_stats(dfTest[:,0],dfTest[:,1],dfTest[:,2],dfTest[:,3],dfTest[:,4],dfTest[:,5],equal_var=False)

In [ ]:
pickle.dump(dfTest, open( "test2.p", "wb" ))

In [59]:
list(keyList)[118]

'0_BMW 6 Series Gran Coupe'

In [69]:
dfTest.describe().values[0,:]

array([ 210.,  210.,  210.,  210.,  210.,  210.,  210.,  210.,  210.,
        210.,  210.,  210.,  210.,  210.,  210.,  210.,  210.,  210.,
        210.,  210.,  210.,  210.,  210.,  210.,  210.,  210.,  210.,
        210.,  210.,  210.,  210.,  210.,  210.,  210.,  210.,  210.,
        210.,  210.,  210.,  210.,  210.,  210.,  210.,  210.,  210.,
        210.,  210.,  210.,  210.,  210.,  210.,  210.,  210.,  210.,
        210.,  210.,  210.,  210.,  210.,  210.,  210.,  210.,  210.,
        210.,  210.,  210.,  210.,  210.,  210.,  210.,  210.,  210.,
        210.,  210.,  210.,  210.,  210.,  210.,  210.,  210.,  210.,
        210.,  210.,  210.,  210.,  210.,  210.,  210.,  210.,  210.,
        210.,  210.,  210.,  210.,  210.,  210.,  210.,  210.,  210.,
        210.,  210.,  210.,  210.,  210.,  210.,  210.,  210.,  210.,
        210.,  210.,  210.,  210.,  210.,  210.,  210.,  210.,  210.,
        210.,  210.,  210.,  210.,  210.,  210.,  210.,  210.,  210.,
        210.,  210.,

In [74]:
#X=dfTest.values
Z = linkage(dfTest, 'ward')
print(is_valid_linkage(Z))

True


In [79]:
# calculate full dendrogram
plt.figure(figsize=(250, 100))
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('sample index')
plt.ylabel('distance')
dendrogram(
    Z,
    leaf_rotation=90.,  # rotates the x axis labels
    leaf_font_size=8.,  # font size for the x axis labels
    labels=list(keyList)
)
plt.show()